<font size="6"> GROUP 14 - Project Proposal </font>

By Eric Zhang, Primo Huang, Edmund Chu, Prajna Nayak

<font size="4"> INTRODUCTION </font>

There are a number of different factors that contribute to one's income. Our multivariable dataset provides information on each individual that we can use to predict the income of an individual. We want to predict if the income of a US citizen is less than or equal 50k, or above 50k.

We will be using the Census Income data set found on https://archive.ics.uci.edu/dataset/20/census+income. This 
data contains 32560 people's information and whether or not their income is under 50,000 or above.

We plan to use variables such as:
1) Age
2) work_class
3) education
4) race
5) sex

and possibly more to see if it is a factor contributing to one's income.

<font size = "4"> PRELIMINARY RESULTS</font>

First we load all the necessary libraries for us to analyze our data

In [10]:
library(tidyverse)
library(repr)
library(tidymodels)
library(ggplot2)

In [11]:
income_col_names <- c("age", "work_class", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "income_status")
income_main <- read_csv("income.csv",col_names=income_col_names)
head(income_main)

Rows: 32561 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): work_class, education, marital_status, occupation, relationship, ra...
dbl (6): age, fnlwgt, education_num, capital_gain, capital_loss, hours_per_week

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,work_class,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_status
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [12]:
income_want <- c("age", "work_class", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country", "income_status")
income = income_main[income_want]

head(income)

age,work_class,education,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income_status
<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K
50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K
37,Private,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,<=50K


Graphs (edmund): 

<font size = "4"> METHODS</font>